In [1]:
import sys
from pathlib import Path
# Get the repository root (1 levels up from current notebook)
repo_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(repo_root))

# import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import Template
# Definición de modulos

import torch
import numpy             as np
import pandas            as pd
import seaborn           as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

from imblearn.over_sampling  import SMOTE, ADASYN
from sklearn.ensemble        import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics         import confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score, recall_score, f1_score, precision_score
from sklearn.decomposition import PCA


Template = Template.Template()

# Build the path to the Excel file in the repository root
data_path = os.path.join(os.path.dirname(os.getcwd()), '..', 'Datos1_InteraccionesNIR.xlsx')

In [5]:
# Build the path to the Excel file in the repository root
data_path = os.path.join(repo_root, 'Datos1_InteraccionesNIR.xlsx')
# import the data from all sheets of the Excel file

df0, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15 = [
    pd.read_excel(data_path, sheet_name=i) for i in range(16)
]


# SMOTE

## Day 1

In [15]:
# Make a new column 'Health_Status'
df = df1.copy()

columns_idx = list(df.columns)[1:]

# Add 'Sana' column at the beginning (after 'Tratamiento' and 'Planta')
df.insert(1, 'Sana', df['Tratamiento'].apply(lambda x: 1 if x == 'Control' else 0))

df.head()



,Tratamiento,Sana,350,351,352,353,354,355,356,357,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,Control,1,0.078084,0.075299,0.077410,0.074002,0.070505,0.067700,0.062893,0.062420,...,0.036214,0.036081,0.035957,0.035807,0.035688,0.035551,0.035364,0.035257,0.035179,0.035142
1,Control,1,0.073550,0.067523,0.071681,0.067482,0.063859,0.062469,0.059024,0.057638,...,0.033057,0.032865,0.032683,0.032480,0.032208,0.032009,0.031834,0.031691,0.031569,0.031492
2,Control,1,0.078702,0.072332,0.076551,0.075096,0.069730,0.063826,0.060950,0.061619,...,0.032032,0.031893,0.031702,0.031533,0.031349,0.031142,0.030905,0.030750,0.030586,0.030511
3,Control,1,0.068565,0.067309,0.070531,0.066035,0.062364,0.060529,0.055741,0.052761,...,0.031268,0.031147,0.030997,0.030795,0.030522,0.030354,0.030158,0.030026,0.030024,0.029915
4,Control,1,0.070898,0.067701,0.075494,0.069948,0.063975,0.061855,0.060048,0.059379,...,0.029287,0.029146,0.029101,0.028976,0.028791,0.028641,0.028453,0.028389,0.028404,0.028371


In [16]:
# Apply SMOTE to balance the dataset
X = df[columns_idx]
y = df['Sana']

smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Create balanced dataframe
df_balanced = pd.DataFrame(X_balanced, columns=columns_idx)
df_balanced.insert(0, 'Sana', y_balanced)

print(f"Original class distribution:\n{y.value_counts()}")
print(f"\nBalanced class distribution:\n{y_balanced.value_counts()}")
print(f"\nOriginal shape: {X.shape}, Balanced shape: {X_balanced.shape}")

df_balanced.head()

Original class distribution:
Sana
0    209
1     30
Name: count, dtype: int64

Balanced class distribution:
Sana
1    209
0    209
Name: count, dtype: int64

Original shape: (239, 2151), Balanced shape: (418, 2151)


C:\Users\ricar\AppData\Local\Temp\ipykernel_14092\2984401751.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_balanced.insert(0, 'Sana', y_balanced)


,Sana,350,351,352,353,354,355,356,357,358,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,1,0.078084,0.075299,0.077410,0.074002,0.070505,0.067700,0.062893,0.062420,0.063808,...,0.036214,0.036081,0.035957,0.035807,0.035688,0.035551,0.035364,0.035257,0.035179,0.035142
1,1,0.073550,0.067523,0.071681,0.067482,0.063859,0.062469,0.059024,0.057638,0.058499,...,0.033057,0.032865,0.032683,0.032480,0.032208,0.032009,0.031834,0.031691,0.031569,0.031492
2,1,0.078702,0.072332,0.076551,0.075096,0.069730,0.063826,0.060950,0.061619,0.062749,...,0.032032,0.031893,0.031702,0.031533,0.031349,0.031142,0.030905,0.030750,0.030586,0.030511
3,1,0.068565,0.067309,0.070531,0.066035,0.062364,0.060529,0.055741,0.052761,0.053525,...,0.031268,0.031147,0.030997,0.030795,0.030522,0.030354,0.030158,0.030026,0.030024,0.029915
4,1,0.070898,0.067701,0.075494,0.069948,0.063975,0.061855,0.060048,0.059379,0.059372,...,0.029287,0.029146,0.029101,0.028976,0.028791,0.028641,0.028453,0.028389,0.028404,0.028371


## Day 2

In [20]:
# Make a new column 'Health_Status'
df = df2.copy()

columns_idx = list(df.columns)[1:]

# Add 'Sana' column at the beginning (after 'Tratamiento' and 'Planta')
df.insert(1, 'Sana', df['Tratamiento'].apply(lambda x: 1 if x == 'Control' else 0))

df.head()

,Tratamiento,Sana,350,351,352,353,354,355,356,357,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,Control,1,0.057346,0.056540,0.052751,0.053504,0.054180,0.052891,0.050285,0.046155,...,0.033542,0.033385,0.033349,0.033178,0.032902,0.032731,0.032548,0.032408,0.032326,0.032270
1,Control,1,0.063527,0.063055,0.062428,0.061098,0.060865,0.060550,0.057198,0.053444,...,0.035794,0.035634,0.035392,0.035225,0.035033,0.034945,0.034993,0.034875,0.034711,0.034583
2,Control,1,0.050986,0.050807,0.050726,0.049559,0.049560,0.048814,0.043256,0.041493,...,0.028711,0.028602,0.028589,0.028482,0.028458,0.028318,0.027975,0.027880,0.027913,0.027835
3,Control,1,0.061819,0.061138,0.058046,0.059413,0.058527,0.055289,0.054370,0.051970,...,0.033141,0.032946,0.032699,0.032572,0.032542,0.032512,0.032608,0.032508,0.032356,0.032179
4,Control,1,0.064970,0.064004,0.061963,0.064850,0.065684,0.062802,0.058909,0.054344,...,0.035396,0.035206,0.034999,0.034813,0.034685,0.034548,0.034402,0.034266,0.034165,0.034061


In [21]:
# Apply SMOTE to balance the dataset
X = df[columns_idx]
y = df['Sana']

smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Create balanced dataframe
df_balanced = pd.DataFrame(X_balanced, columns=columns_idx)
df_balanced.insert(0, 'Sana', y_balanced)

print(f"Original class distribution:\n{y.value_counts()}")
print(f"\nBalanced class distribution:\n{y_balanced.value_counts()}")
print(f"\nOriginal shape: {X.shape}, Balanced shape: {X_balanced.shape}")

df_balanced.head()

Original class distribution:
Sana
0    205
1     28
Name: count, dtype: int64

Balanced class distribution:
Sana
1    205
0    205
Name: count, dtype: int64

Original shape: (233, 2151), Balanced shape: (410, 2151)


C:\Users\ricar\AppData\Local\Temp\ipykernel_14092\2984401751.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_balanced.insert(0, 'Sana', y_balanced)


,Sana,350,351,352,353,354,355,356,357,358,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,1,0.057346,0.056540,0.052751,0.053504,0.054180,0.052891,0.050285,0.046155,0.044742,...,0.033542,0.033385,0.033349,0.033178,0.032902,0.032731,0.032548,0.032408,0.032326,0.032270
1,1,0.063527,0.063055,0.062428,0.061098,0.060865,0.060550,0.057198,0.053444,0.051902,...,0.035794,0.035634,0.035392,0.035225,0.035033,0.034945,0.034993,0.034875,0.034711,0.034583
2,1,0.050986,0.050807,0.050726,0.049559,0.049560,0.048814,0.043256,0.041493,0.040846,...,0.028711,0.028602,0.028589,0.028482,0.028458,0.028318,0.027975,0.027880,0.027913,0.027835
3,1,0.061819,0.061138,0.058046,0.059413,0.058527,0.055289,0.054370,0.051970,0.049780,...,0.033141,0.032946,0.032699,0.032572,0.032542,0.032512,0.032608,0.032508,0.032356,0.032179
4,1,0.064970,0.064004,0.061963,0.064850,0.065684,0.062802,0.058909,0.054344,0.053378,...,0.035396,0.035206,0.034999,0.034813,0.034685,0.034548,0.034402,0.034266,0.034165,0.034061


## Day 3

In [22]:
# Make a new column 'Health_Status'
df = df3.copy()

columns_idx = list(df.columns)[1:]

# Add 'Sana' column at the beginning (after 'Tratamiento' and 'Planta')
df.insert(1, 'Sana', df['Tratamiento'].apply(lambda x: 1 if x == 'Control' else 0))

df.head()

,Tratamiento,Sana,350,351,352,353,354,355,356,357,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,Control,1,0.077096,0.076401,0.077640,0.073403,0.069708,0.067420,0.063497,0.063387,...,0.028871,0.028704,0.028565,0.028401,0.028233,0.028111,0.027981,0.027884,0.027915,0.027741
1,Control,1,0.074623,0.074337,0.075459,0.069064,0.065403,0.065427,0.062845,0.060349,...,0.030656,0.030465,0.030290,0.030118,0.030026,0.029918,0.029792,0.029630,0.029490,0.029318
2,Control,1,0.068844,0.064497,0.067699,0.062913,0.059612,0.059234,0.056685,0.054846,...,0.030536,0.030366,0.030277,0.030151,0.030064,0.029900,0.029709,0.029502,0.029186,0.029041
3,Control,1,0.066975,0.064955,0.066634,0.061690,0.059538,0.059540,0.053983,0.052394,...,0.025271,0.025113,0.024971,0.024842,0.024703,0.024573,0.024487,0.024374,0.024245,0.024119
4,Control,1,0.067311,0.064306,0.067247,0.061632,0.058040,0.056864,0.051009,0.050021,...,0.026281,0.026183,0.026014,0.025855,0.025573,0.025490,0.025571,0.025451,0.025312,0.025232


In [23]:
# Apply SMOTE to balance the dataset
X = df[columns_idx]
y = df['Sana']

smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Create balanced dataframe
df_balanced = pd.DataFrame(X_balanced, columns=columns_idx)
df_balanced.insert(0, 'Sana', y_balanced)

print(f"Original class distribution:\n{y.value_counts()}")
print(f"\nBalanced class distribution:\n{y_balanced.value_counts()}")
print(f"\nOriginal shape: {X.shape}, Balanced shape: {X_balanced.shape}")

df_balanced.head()

Original class distribution:
Sana
0    207
1     30
Name: count, dtype: int64

Balanced class distribution:
Sana
1    207
0    207
Name: count, dtype: int64

Original shape: (237, 2151), Balanced shape: (414, 2151)


C:\Users\ricar\AppData\Local\Temp\ipykernel_14092\2984401751.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_balanced.insert(0, 'Sana', y_balanced)


,Sana,350,351,352,353,354,355,356,357,358,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,1,0.077096,0.076401,0.077640,0.073403,0.069708,0.067420,0.063497,0.063387,0.064254,...,0.028871,0.028704,0.028565,0.028401,0.028233,0.028111,0.027981,0.027884,0.027915,0.027741
1,1,0.074623,0.074337,0.075459,0.069064,0.065403,0.065427,0.062845,0.060349,0.060482,...,0.030656,0.030465,0.030290,0.030118,0.030026,0.029918,0.029792,0.029630,0.029490,0.029318
2,1,0.068844,0.064497,0.067699,0.062913,0.059612,0.059234,0.056685,0.054846,0.054258,...,0.030536,0.030366,0.030277,0.030151,0.030064,0.029900,0.029709,0.029502,0.029186,0.029041
3,1,0.066975,0.064955,0.066634,0.061690,0.059538,0.059540,0.053983,0.052394,0.052874,...,0.025271,0.025113,0.024971,0.024842,0.024703,0.024573,0.024487,0.024374,0.024245,0.024119
4,1,0.067311,0.064306,0.067247,0.061632,0.058040,0.056864,0.051009,0.050021,0.052026,...,0.026281,0.026183,0.026014,0.025855,0.025573,0.025490,0.025571,0.025451,0.025312,0.025232


## Day 4

In [24]:
# Make a new column 'Health_Status'
df = df4.copy()

columns_idx = list(df.columns)[1:]

# Add 'Sana' column at the beginning (after 'Tratamiento' and 'Planta')
df.insert(1, 'Sana', df['Tratamiento'].apply(lambda x: 1 if x == 'Control' else 0))

df.head()

,Tratamiento,Sana,350,351,352,353,354,355,356,357,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,Control,1,0.074723,0.074907,0.069226,0.073145,0.069501,0.060720,0.062748,0.060955,...,0.039735,0.039615,0.039570,0.039455,0.039427,0.039334,0.039157,0.038921,0.038661,0.038498
1,Control,1,0.064413,0.063294,0.060044,0.061458,0.059083,0.053877,0.053428,0.052120,...,0.037456,0.037284,0.037252,0.037116,0.036860,0.036708,0.036610,0.036515,0.036418,0.036374
2,Control,1,0.065758,0.065233,0.060532,0.060233,0.059288,0.056705,0.054850,0.051916,...,0.038584,0.038430,0.038355,0.038257,0.038215,0.038105,0.038004,0.037822,0.037529,0.037371
3,Control,1,0.062238,0.062061,0.057599,0.059343,0.057833,0.052907,0.051349,0.048695,...,0.036916,0.036795,0.036624,0.036527,0.036393,0.036301,0.036340,0.036221,0.035973,0.035830
4,Control,1,0.084853,0.082573,0.083905,0.081276,0.074833,0.069509,0.072085,0.071686,...,0.040126,0.039897,0.039598,0.039449,0.039432,0.039265,0.039116,0.038979,0.038711,0.038569


In [ ]:
# Apply SMOTE to balance the dataset
X = df[columns_idx]
y = df['Sana']

smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Create balanced dataframe
df_balanced = pd.DataFrame(X_balanced, columns=columns_idx)
df_balanced.insert(0, 'Sana', y_balanced)

print(f"Original class distribution:\n{y.value_counts()}")
print(f"\nBalanced class distribution:\n{y_balanced.value_counts()}")
print(f"\nOriginal shape: {X.shape}, Balanced shape: {X_balanced.shape}")

df_balanced.head()